# Importing Data and Packages #

In [1]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [12]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from precision_recall_cutoff import precision_recall_cutoff

from tqdm import tqdm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier 
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, recall_score
from sklearn.feature_selection import RFE, RFECV

pd.set_option('display.max_columns', 50)

store_final = pd.read_csv('store_final.csv')
store_final.head()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Income_imp,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Widow,Age,Child,MntTotal,Recency_u50,TotalPurchses,Enr_year,Enr_quarter,Enr_month,Log_Income,Interaction_1,Interaction_2,Interaction_3,Interaction_4,Interaction_5,Interaction_6,Tree_1,Tree_2,Tree_3,Response
0,1970,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,1,0,0,1,0,0,0,53,0,1190,1,14,2014,2,6,11.348463,259420,4760,0,872,0,0,0,0,0,1
1,1961,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,1,0,0,0,0,1,0,62,0,577,1,17,2014,2,6,10.952402,21349,1731,0,111,0,0,0,1,0,1
2,1958,67267.0,0,1,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,1,0,0,0,1,0,0,65,1,251,1,10,2014,2,5,11.116425,7530,502,0,60,0,0,0,1,0,0
3,1967,32474.0,1,1,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,1,0,0,0,1,0,0,56,1,11,1,3,2014,4,11,10.388195,0,0,0,0,0,0,0,1,0,0
4,1989,21474.0,1,0,0,6,16,24,11,0,34,2,3,1,2,7,0,0,0,1,0,0,0,0,1,0,34,1,91,1,6,2014,3,8,9.974598,3094,91,0,34,0,0,0,1,0,1


# Hyper-Parameter Tuning #

In [13]:
#defining input (top 8) and target (Response)
x=store_final[['Recency', 'MntTotal', 'Interaction_2', 'Enr_year', 'MntWines', 'Interaction_4', 'MntMeatProducts', 'NumWebVisitsMonth']]
y=store_final['Response']

#splitting the data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

In [14]:
#defining top 6 and 7 variable dataframes for x_train and x_test
x_train_7 = x_train.drop(columns = 'NumWebVisitsMonth')
x_test_7 = x_test.drop(columns = 'NumWebVisitsMonth')

x_train_6 = x_train_7.drop(columns = 'MntMeatProducts')
x_test_6 = x_test_7.drop(columns = 'MntMeatProducts')

**Decision Tree**

*top 8 features*

In [15]:
#defining parameter dictionary
dt_param_grid = {'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

#running GridSearchCV
dt_grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), 
                               param_grid = dt_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train, y_train)

#extracting best hyperparameter combination
dt_grid_search.best_params_

{'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 10}

In [16]:
#building the model with best hyperparameters
dt_md = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7).fit(x_train, y_train)

#predicting on test
dt_pred = dt_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
dt_label = precision_recall_cutoff(y_test, dt_pred)

#print classification report
print(classification_report(y_test, dt_label))
print('recall:',recall_score(y_test, dt_label))

              precision    recall  f1-score   support

           0       0.86      0.72      0.79       382
           1       0.76      0.88      0.82       381

    accuracy                           0.80       763
   macro avg       0.81      0.80      0.80       763
weighted avg       0.81      0.80      0.80       763

recall: 0.884514435695538


*top 7 features*

In [17]:
#defining parameter dictionary
dt_param_grid = {'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

#running GridSearchCV
dt_grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), 
                               param_grid = dt_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_7, y_train)

#extracting best hyperparameter combination
dt_grid_search.best_params_

{'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 10}

In [18]:
#building the model with best hyperparameters
dt_md = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7).fit(x_train_7, y_train)

#predicting on test
dt_pred = dt_md.predict_proba(x_test_7)[:,1]

#changing likelihood to labels
dt_label = precision_recall_cutoff(y_test, dt_pred)

#print classification report
print(classification_report(y_test, dt_label))
print('recall:',recall_score(y_test, dt_label))

              precision    recall  f1-score   support

           0       0.84      0.71      0.77       382
           1       0.75      0.86      0.80       381

    accuracy                           0.79       763
   macro avg       0.79      0.79      0.78       763
weighted avg       0.79      0.79      0.78       763

recall: 0.8608923884514436


*top 6 features*

In [19]:
#defining parameter dictionary
dt_param_grid = {'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

#running GridSearchCV
dt_grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), 
                               param_grid = dt_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_6, y_train)

#extracting best hyperparameter combination
dt_grid_search.best_params_

{'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 10}

In [20]:
#building the model with best hyperparameters
dt_md = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 7).fit(x_train_6, y_train)

#predicting on test
dt_pred = dt_md.predict_proba(x_test_6)[:,1]

#changing likelihood to labels
dt_label = precision_recall_cutoff(y_test, dt_pred)

#print classification report
print(classification_report(y_test, dt_label))
print('recall:',recall_score(y_test, dt_label))

              precision    recall  f1-score   support

           0       0.83      0.76      0.79       382
           1       0.78      0.84      0.81       381

    accuracy                           0.80       763
   macro avg       0.80      0.80      0.80       763
weighted avg       0.80      0.80      0.80       763

recall: 0.84251968503937


________
Best model: top 8 features with {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 10}

**Random Forest**

*top 8 features*

In [25]:
#defining parameter dictionary
rf_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth' : [3, 5, 7]}

#running GridSearchCV
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rf_param_grid, 
                              cv = 3, scoring = 'recall').fit(x_train,y_train)

#extracting best hyperparameter combination
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 300}

In [26]:
#building the model with best hyperparameters
rf_md = RandomForestClassifier(**rf_grid_search.best_params_).fit(x_train,y_train)

#predicting on test
rf_pred = rf_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
rf_label = precision_recall_cutoff(y_test, rf_pred)

#print classification report
print(classification_report(y_test,rf_label))
print('recall:',recall_score(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.88      0.84      0.86       382
           1       0.85      0.88      0.87       381

    accuracy                           0.86       763
   macro avg       0.86      0.86      0.86       763
weighted avg       0.86      0.86      0.86       763

recall: 0.884514435695538


*top 7 features*

In [27]:
#defining parameter dictionary
rf_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth' : [3, 5, 7]}

#running GridSearchCV
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rf_param_grid, 
                              cv = 3, scoring = 'recall').fit(x_train_7,y_train)

#extracting best hyperparameter combination
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 15,
 'n_estimators': 500}

In [28]:
#building the model with best hyperparameters
rf_md = RandomForestClassifier(**rf_grid_search.best_params_).fit(x_train_7,y_train)

#predicting on test
rf_pred = rf_md.predict_proba(x_test_7)[:,1]

#changing likelihood to labels
rf_label = precision_recall_cutoff(y_test, rf_pred)

#print classification report
print(classification_report(y_test,rf_label))
print('recall:',recall_score(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85       382
           1       0.84      0.87      0.86       381

    accuracy                           0.85       763
   macro avg       0.85      0.85      0.85       763
weighted avg       0.85      0.85      0.85       763

recall: 0.8740157480314961


*top 6 features*

In [29]:
#defining parameter dictionary
rf_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth' : [3, 5, 7]}

#running GridSearchCV
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rf_param_grid, 
                              cv = 3, scoring = 'recall').fit(x_train_6,y_train)

#extracting best hyperparameter combination
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 300}

In [30]:
#building the model with best hyperparameters
rf_md = RandomForestClassifier(**rf_grid_search.best_params_).fit(x_train_6,y_train)

#predicting on test
rf_pred = rf_md.predict_proba(x_test_6)[:,1]

#changing likelihood to labels
rf_label = precision_recall_cutoff(y_test, rf_pred)

#print classification report
print(classification_report(y_test,rf_label))
print('recall:',recall_score(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.88      0.83      0.85       382
           1       0.84      0.89      0.86       381

    accuracy                           0.86       763
   macro avg       0.86      0.86      0.86       763
weighted avg       0.86      0.86      0.86       763

recall: 0.889763779527559


________
Best model: top 6 features with {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 300}

**AdaBoost**

*top 8 features*

In [32]:
#defining parameter dictionary
ada_param_grid = {'n_estimators': [100, 300, 500],
                  'estimator__min_samples_split': [10, 15],
                  'estimator__min_samples_leaf': [5, 7],
                  'estimator__max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
ada_grid_search = GridSearchCV(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier()), 
                               param_grid = ada_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train, y_train)

#extracting best hyperparameter combination
ada_grid_search.best_params_

{'estimator__max_depth': 3,
 'estimator__min_samples_leaf': 7,
 'estimator__min_samples_split': 10,
 'learning_rate': 0.1,
 'n_estimators': 300}

In [35]:
#building the model with best hyperparameters
ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 7, max_depth = 3),
                            n_estimators = 300, learning_rate = 0.1).fit(x_train, y_train)

#predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
ada_label = precision_recall_cutoff(y_test, ada_pred)

#print classification report
print(classification_report(y_test, ada_label))
print('recall:',recall_score(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       382
           1       0.92      0.97      0.94       381

    accuracy                           0.94       763
   macro avg       0.94      0.94      0.94       763
weighted avg       0.94      0.94      0.94       763

recall: 0.9658792650918635


*top 7 features*

In [36]:
#defining parameter dictionary
ada_param_grid = {'n_estimators': [100, 300, 500],
                  'estimator__min_samples_split': [10, 15],
                  'estimator__min_samples_leaf': [5, 7],
                  'estimator__max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
ada_grid_search = GridSearchCV(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier()), 
                               param_grid = ada_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_7, y_train)

#extracting best hyperparameter combination
ada_grid_search.best_params_

{'estimator__max_depth': 3,
 'estimator__min_samples_leaf': 5,
 'estimator__min_samples_split': 15,
 'learning_rate': 0.1,
 'n_estimators': 300}

In [37]:
#building the model with best hyperparameters
ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 5, max_depth = 3),
                            n_estimators = 300, learning_rate = 0.1).fit(x_train_7, y_train)

#predicting on test
ada_pred = ada_md.predict_proba(x_test_7)[:,1]

#changing likelihood to labels
ada_label = precision_recall_cutoff(y_test, ada_pred)

#print classification report
print(classification_report(y_test, ada_label))
print('recall:',recall_score(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       382
           1       0.92      0.96      0.94       381

    accuracy                           0.94       763
   macro avg       0.94      0.94      0.94       763
weighted avg       0.94      0.94      0.94       763

recall: 0.963254593175853


*top 6 features*

In [38]:
#defining parameter dictionary
ada_param_grid = {'n_estimators': [100, 300, 500],
                  'estimator__min_samples_split': [10, 15],
                  'estimator__min_samples_leaf': [5, 7],
                  'estimator__max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
ada_grid_search = GridSearchCV(estimator = AdaBoostClassifier(estimator = DecisionTreeClassifier()), 
                               param_grid = ada_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_6, y_train)

#extracting best hyperparameter combination
ada_grid_search.best_params_

{'estimator__max_depth': 3,
 'estimator__min_samples_leaf': 5,
 'estimator__min_samples_split': 10,
 'learning_rate': 0.1,
 'n_estimators': 300}

In [39]:
#building the model with best hyperparameters
ada_md = AdaBoostClassifier(estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 5, max_depth = 3),
                            n_estimators = 300, learning_rate = 0.1).fit(x_train_6, y_train)

#predicting on test
ada_pred = ada_md.predict_proba(x_test_6)[:,1]

#changing likelihood to labels
ada_label = precision_recall_cutoff(y_test, ada_pred)

#print classification report
print(classification_report(y_test, ada_label))
print('recall:',recall_score(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93       382
           1       0.91      0.96      0.94       381

    accuracy                           0.93       763
   macro avg       0.94      0.93      0.93       763
weighted avg       0.94      0.93      0.93       763

recall: 0.963254593175853


________
Best model: top 8 features with {'estimator__max_depth': 3,'estimator__min_samples_leaf': 7,'estimator__min_samples_split': 10,'learning_rate': 0.1,'n_estimators': 300}

**Gradient Boosting**

*top 8 features*

In [40]:
#defining parameter dictionary
gb_param_grid = {'n_estimators': [100, 300, 500],
                  'min_samples_split': [10, 15],
                  'min_samples_leaf': [5, 7],
                  'max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
gb_grid_search = GridSearchCV(estimator = GradientBoostingClassifier(), 
                               param_grid = gb_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train, y_train)

#extracting best hyperparameter combination
gb_grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 100}

In [41]:
#building the model with best hyperparameters
gb_md = GradientBoostingClassifier(**gb_grid_search.best_params_).fit(x_train, y_train)

#predicting on test
gb_pred = gb_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
gb_label = precision_recall_cutoff(y_test, gb_pred)

#print classification report
print(classification_report(y_test, gb_label))
print('recall:',recall_score(y_test, gb_label))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       382
           1       0.95      0.97      0.96       381

    accuracy                           0.96       763
   macro avg       0.96      0.96      0.96       763
weighted avg       0.96      0.96      0.96       763

recall: 0.973753280839895


*top 7 features*

In [42]:
#defining parameter dictionary
gb_param_grid = {'n_estimators': [100, 300, 500],
                  'min_samples_split': [10, 15],
                  'min_samples_leaf': [5, 7],
                  'max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
gb_grid_search = GridSearchCV(estimator = GradientBoostingClassifier(), 
                               param_grid = gb_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_7, y_train)

#extracting best hyperparameter combination
gb_grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 15,
 'n_estimators': 500}

In [43]:
#building the model with best hyperparameters
gb_md = GradientBoostingClassifier(**gb_grid_search.best_params_).fit(x_train_7, y_train)

#predicting on test
gb_pred = gb_md.predict_proba(x_test_7)[:,1]

#changing likelihood to labels
gb_label = precision_recall_cutoff(y_test, gb_pred)

#print classification report
print(classification_report(y_test, gb_label))
print('recall:',recall_score(y_test, gb_label))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       382
           1       0.98      0.93      0.96       381

    accuracy                           0.96       763
   macro avg       0.96      0.96      0.96       763
weighted avg       0.96      0.96      0.96       763

recall: 0.9343832020997376


*top 6 features*

In [46]:
#defining parameter dictionary
gb_param_grid = {'n_estimators': [100, 300, 500],
                  'min_samples_split': [10, 15],
                  'min_samples_leaf': [5, 7],
                  'max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1]}

#running GridSearchCV
gb_grid_search = GridSearchCV(estimator = GradientBoostingClassifier(), 
                               param_grid = gb_param_grid, 
                               cv = 3, 
                               scoring = 'recall',
                               n_jobs = -1).fit(x_train_6, y_train)

#extracting best hyperparameter combination
gb_grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 300}

In [47]:
#building the model with best hyperparameters
gb_md = GradientBoostingClassifier(**gb_grid_search.best_params_).fit(x_train_6, y_train)

#predicting on test
gb_pred = gb_md.predict_proba(x_test_6)[:,1]

#changing likelihood to labels
gb_label = precision_recall_cutoff(y_test, gb_pred)

#print classification report
print(classification_report(y_test, gb_label))
print('recall:',recall_score(y_test, gb_label))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       382
           1       0.94      0.97      0.95       381

    accuracy                           0.95       763
   macro avg       0.95      0.95      0.95       763
weighted avg       0.95      0.95      0.95       763

recall: 0.9658792650918635


________
Best model: top 8 features with 
{'learning_rate': 0.1,
 'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 100}

**XGBoost**

*top 8 features*

In [48]:
#defining parameter dictionary
xgb_param_grid = {'n_estimators': [100, 300, 500],
                  'max_depth': [3, 5, 7],
                  'min_child_weight': [5, 7],
                  'learning_rate': [0.01, 0.1],
                  'gamma': [0.3, 0.1],
                  'subsample': [0.8, 1],
                  'colsample_bytree': [1]}

#running GridSearchCV
xgb_grid_search = GridSearchCV(estimator = XGBClassifier(), param_grid = xgb_param_grid, 
                              cv = 3, scoring = 'recall', n_jobs = -1).fit(x_train,y_train)

#extracting best hyperparameter combination
xgb_grid_search.best_params_

{'colsample_bytree': 1,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

In [49]:
#building the model with best hyperparameters
xgb_md = XGBClassifier(**xgb_grid_search.best_params_).fit(x_train, y_train)

#predicting on test
xgb_pred = xgb_md.predict_proba(x_test)[:,1]

#changing likelihood to labels
xgb_label = precision_recall_cutoff(y_test, xgb_pred)

#print classification report
print(classification_report(y_test, xgb_label))
print('recall:',recall_score(y_test, xgb_label))

              precision    recall  f1-score   support

           0       0.98      0.93      0.96       382
           1       0.94      0.98      0.96       381

    accuracy                           0.96       763
   macro avg       0.96      0.96      0.96       763
weighted avg       0.96      0.96      0.96       763

recall: 0.9816272965879265


*top 7 features*

In [50]:
#defining parameter dictionary
xgb_param_grid = {'n_estimators': [100, 300, 500],
                  'max_depth': [3, 5, 7],
                  'min_child_weight': [5, 7],
                  'learning_rate': [0.01, 0.1],
                  'gamma': [0.3, 0.1],
                  'subsample': [0.8, 1],
                  'colsample_bytree': [1]}

#running GridSearchCV
xgb_grid_search = GridSearchCV(estimator = XGBClassifier(), param_grid = xgb_param_grid, 
                              cv = 3, scoring = 'recall', n_jobs = -1).fit(x_train_7,y_train)

#extracting best hyperparameter combination
xgb_grid_search.best_params_

{'colsample_bytree': 1,
 'gamma': 0.3,
 'learning_rate': 0.1,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

In [51]:
#building the model with best hyperparameters
xgb_md = XGBClassifier(**xgb_grid_search.best_params_).fit(x_train_7, y_train)

#predicting on test
xgb_pred = xgb_md.predict_proba(x_test_7)[:,1]

#changing likelihood to labels
xgb_label = precision_recall_cutoff(y_test, xgb_pred)

#print classification report
print(classification_report(y_test, xgb_label))
print('recall:',recall_score(y_test, xgb_label))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       382
           1       0.95      0.96      0.95       381

    accuracy                           0.95       763
   macro avg       0.95      0.95      0.95       763
weighted avg       0.95      0.95      0.95       763

recall: 0.9606299212598425


*top 6 features*

In [52]:
#defining parameter dictionary
xgb_param_grid = {'n_estimators': [100, 300, 500],
                  'max_depth': [3, 5, 7],
                  'min_child_weight': [5, 7],
                  'learning_rate': [0.01, 0.1],
                  'gamma': [0.3, 0.1],
                  'subsample': [0.8, 1],
                  'colsample_bytree': [1]}

#running GridSearchCV
xgb_grid_search = GridSearchCV(estimator = XGBClassifier(), param_grid = xgb_param_grid, 
                              cv = 3, scoring = 'recall', n_jobs = -1).fit(x_train_6,y_train)

#extracting best hyperparameter combination
xgb_grid_search.best_params_

{'colsample_bytree': 1,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

In [54]:
#building the model with best hyperparameters
xgb_md = XGBClassifier(**xgb_grid_search.best_params_).fit(x_train_6, y_train)

#predicting on test
xgb_pred = xgb_md.predict_proba(x_test_6)[:,1]

#changing likelihood to labels
xgb_label = precision_recall_cutoff(y_test, xgb_pred)

#print classification report
print(classification_report(y_test, xgb_label))
print('recall:',recall_score(y_test, xgb_label))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93       382
           1       0.92      0.95      0.94       381

    accuracy                           0.93       763
   macro avg       0.93      0.93      0.93       763
weighted avg       0.93      0.93      0.93       763

recall: 0.9501312335958005


________
Best model: top 8 features with {'colsample_bytree': 1,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}